In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    llm = ChatOpenAI(model_name=MODEL_NAME)
    llm_with_tools = llm.bind_tools(tools)

    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    graph_builder = StateGraph(State)
    graph_builder.add_node("chatbot", chatbot)

    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)

    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )

    graph_builder.add_edge("tools", "chatbot")
    graph_builder.set_entry_point("chatbot")

    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)
    return graph

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述    
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述
graph = build_graph(model_name=MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    print("質問:", user_input, flush=True)
    stream_graph_updates(graph, user_input)

質問: こんにちは！
こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？
質問: 1足す2は？
1足す2は？
1足す2は3です。何か他に知りたいことはありますか？
質問: 台湾観光について検索結果を教えて
台湾観光について検索結果を教えて

[{"url": "https://www.hankyu-travel.com/guide/taiwan/", "content": "# 台湾観光におすすめの名所＆人気のスポットランキング台湾観光ガイド. ## 台湾観光マップ. ## 台湾のおすすめ観光スポット総合ランキング. ### 九フン. ### 士林夜市. ### 日月潭. ### 台北101. ### 野柳. ### 龍虎塔. ### 太魯閣溪谷. ## 台湾のおすすめ観光スポット. #### テーマで絞り込む. #### エリアで絞り込む. 鼎泰豐は小籠包をはじめとした点心を中心に台湾料理を提供する、台湾を代表するレストランです。世界中に支店があり、2010年には香港ミシュランガイドで一つ星を獲得するなど、小籠包といえば鼎泰豐と言っても過言ではありません。そんな世界中で味わえる鼎泰豐の小籠包ですが、やはり本場台湾で食べる小籠包などの点心は別格の一言。お店の活気や雰囲気、サービスレベルの高さと合わせて、より一層美味しくいただけます。また台湾各地に支店があり、比較的カジュアルな服装でも入店できるため、気取らずに気軽に小籠包などが楽しめるのもポイントです。台湾に来たなら、ぜひ一度は訪れておきたいレストランです。. 赤レンガのレトロな外観が印象的な宮原眼科。眼科と間違えてしまいそうな名前ですが、日本統治時代に宮原武熊氏が開設した眼科診療所があった建物を、パイナップルケーキで有名な日出がリノベーションしたスイーツショップです。中に入れば1階から４階にかけて広がる吹き抜けの空間があり、重厚感あふれる建築様式と合わせて、まるでハリーポッターの魔法学校に迷い込んだかのよう。非日常的な空間を楽しみながら、パイナップルケーキなどが選べます。特に60種類以上のフレーバーとさまざまなトッピングがある宮原アイスクリームはインスタ映え間違いなしの可愛らしさ。もちろん味も美味しく、台湾に来たらぜひ一度は訪ねておきたいスイーツショップです。. 基隆中正公園は、基隆の港と街並みを一望